In [11]:
import numpy as np

from tqdm import tqdm

from autograd import elementwise_grad as egrad
from autograd import jacobian
import autograd.numpy as np_

import matplotlib.pyplot as plt

In [4]:
m = 1
n = 3
Tf = 100 
Delta_T = 0.0001

TimeSpan = np.arange(0, Tf, Delta_T)
nt = TimeSpan.shape[0] # mt do not use in python 

u = np.zeros((nt,m))
x = np.zeros((nt,n))

In [5]:
for j in range(n):
    x[0][j] = 2*(j+1)

A_s = 0.001 * np.diag([-5,-7,-6])
B = np.array([0, 0, 1])#.reshape(-3, 1)

for i in range(nt-1):
    u[i] = 500 * np.sin(0.00004*i) * np.cos(0.000009*i+1.15)

    x[i+1][:] = x[i][:] + Delta_T * ( np.matmul(A_s, x[i][:]) + B * u[i] + np.random.random_sample(3) * np.array([70, 80, 90]) )

In [6]:
C_ = 0.1 * np.array([1, 1, 1]).reshape(-3,1)

def sigma_1(x):
    return (1 / (1 + 0.22 * np_.exp(-0.2 * np_.matmul(x, C_))) - 0.3)

def sigma_2(x):
    return (1 / (1 + 0.2 * np_.exp(-0.2 * np_.matmul(x, C_))) - 0.2)

def calc_Hes(sigma, x):
    H_sigma = jacobian(egrad(sigma))
    length = x.shape[0]
    H_arr = np.full(x.shape[0] * 9, -1.0).reshape(-3, 3, 3)

    
    for i in tqdm(range(length)):
        H = H_sigma(x[i])

        # H_arr[i][:] = H[:] - check result
        H_arr[i,:,:] = H[:,:]

    return H_arr

def calc_norms(Hessians):

    length = Hessians.shape[0]
    norms = np.ones(length) 

    for i in range(length):
        norm = np.linalg.norm(Hessians[i])
        norms[i] = norm 

    return norms

def max_H(Hessians, norms):
    max_index = np.argmax(norms)
    
    return Hessians[max_index]

In [7]:
u = np.zeros((nt,m))
x = np.zeros((nt,n))

In [17]:
#hes_1 = find_Hes(sigma_1, x)
#np.save('hes_1.npy', hes_1)
hes_1 = np.load('hes_1.npy')

In [20]:
#hes_2 = find_Hes(sigma_2, x)
#np.save('hes_2.npy', hes_2) 
hes_2 = np.load('hes_2.npy')

In [21]:
norms_1 = calc_norms(hes_1)
norms_2 = calc_norms(hes_2)

In [22]:
est_1 = max_H(hes_1, norms_1)
est_2 = max_H(hes_2, norms_2)

In [23]:
est_1, est_2

(array([[-3.54624207e-05, -3.54624207e-05, -3.54624207e-05],
        [-3.54624207e-05, -3.54624207e-05, -3.54624207e-05],
        [-3.54624207e-05, -3.54624207e-05, -3.54624207e-05]]),
 array([[-3.4209953e-05, -3.4209953e-05, -3.4209953e-05],
        [-3.4209953e-05, -3.4209953e-05, -3.4209953e-05],
        [-3.4209953e-05, -3.4209953e-05, -3.4209953e-05]]))